In [1]:
!pip install minsearch




[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import minsearch
from openai import OpenAI


In [3]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)



In [5]:
q = 'the course already started, can i still enroll?'

In [6]:
index.fit(documents)

In [7]:
boost = {'question': 3.0, 'section': 0.5}

index.search(
    query= q,
    boost_dict = boost,
    num_results=5
)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [9]:
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=d_key,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="deepseek/deepseek-r1-0528:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

TypeError: 'NoneType' object is not subscriptable

In [49]:
completion = client.chat.completions.create(
  extra_body={},
  model="deepseek/deepseek-r1-0528:free",
  messages=[
    {
      "role": "user",
      "content": q
    }
  ]
)
print(completion.choices[0].message.content)

**Yes, it's often possible to enroll after a course has started, but you need to act quickly! Here's what to do and consider:**

### ✅ **Steps to Take:**
1. **Contact the Instructor/Administrator Immediately:**
   - **For academic courses:** Email the professor/instructor and the registrar or department admin. Ask:  
     *"Is late enrollment still possible? I'm willing to catch up on missed work."*
   - **For online platforms (Coursera, Udemy, etc.):** Most allow self-paced enrollment—just join and start (check the course page).
   - **For bootcamps/workshops:** Email the organizer—flexibility varies.

2. **Explain Your Situation Briefly:**
   - Mention *why* you’re interested and confirm you’re committed to catching up.

3. **Ask Key Questions:**
   - "What’s the latest enrollment date?"
   - "Are there fees for late registration?"
   - "Can I access materials from earlier sessions?"
   - "Are assignments/exams flexible for late enrollees?"

### ⚠️ **Important Considerations:**
- **D

In [50]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results


In [51]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [52]:
def llm(prompt):
    response = client.chat.completions.create(
        extra_body={},
        model="deepseek/deepseek-r1-0528:free",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return response.choices[0].message.content

In [53]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
rag(query)


"Based solely on the provided FAQ context, here's how to run Kafka components in specific scenarios:\n\n1.  **To run a Java Kafka producer/consumer/KStreams application from the terminal:**\n    *   Navigate to the project directory.\n    *   Run:\n        ```bash\n        java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n        ```\n    *   (Replace `<jar_name>` with your actual JAR filename).\n\n2.  **To run Python Kafka scripts and avoid common issues:**\n    *   **Resolve `Module “kafka” not found`:** Create and activate a virtual environment:\n        ```bash\n        python -m venv env          # Create (run once)\n        source env/bin/activate      # Activate (Linux/MacOS)\n        # OR env\\Scripts\\activate   # Activate (Windows)\n        pip install -r ../requirements.txt  # Install dependencies\n        ```\n        Run your Python files (e.g., `producer.py`) within this activated environment. Remember to activate it each tim

## RAG with vector search

In [10]:
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance


/Users/eduacapel/PycharmProjects/LLM-Zoomcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
EMBEDDING_DIMENSIONALITY =512
model_handle = "jinaai/jina-embeddings-v2-small-en"

collection_name = "zoomcamp-faq"

In [ ]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [58]:
points = []
id = 0

for i, doc in enumerate(documents):
    text = doc['question'] + " " + doc['text']
    vector = models.Document(text = text, model = model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

    id += 1

In [59]:
points[0]

PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with ann

In [61]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
query = 'I just discover the course, can I still join'

In [70]:
course = 'data-engineering-zoomcamp'

query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(  # Embed the query text locally
        text=query,
        model=model_handle
    ),
    query_filter=models.Filter(
        must=[
            models.FieldCondition(
                key='course',
                match=models.MatchValue(value=course)  # Fixed typo: "MAtchValue" → "MatchValue"
            )
        ]
    ),
    limit=5,  # Top 5 closest matches
    with_payload=True  # Return metadata in results
)


In [68]:
results = [ ]
for point in query_points.points:
    results.append(point.payload)

In [69]:
results

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'For example, when running JsonConsumer.java, got:\nConsuming form kafka started\nRESULTS:::0\nRESULTS:::0\nRESULTS:::0\nOr when running JsonProducer.java, got:\nException in thread "main" java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.SaslAuthenticationException: Authentication failed\nSolution:\nMake sure in the scripts in src/main/java/org/example/ that you are running (e.g. JsonConsumer.java, JsonProducer.java), the StreamsConfig.BOOTSTRAP_SERVERS_CONFIG is the correct server url (e.g. europe-west3 from example vs europe-west2)\nMake sure cluster key and secrets are updated in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KA

In [71]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
## Homework

In [15]:
from transformers import AutoTokenizer, AutoModel
import torch


model_name = "jinaai/jina-embeddings-v2-small-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

text = "I just discovered the course. Can I join now?"

inputs = tokenizer(text, return_tensors="pt", truncation=True)
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0, :]

embedding_vector = embeddings[0].numpy()
print(embedding_vector.shape)
print(embedding_vector.min())

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

(512,)
-2.6467352


In [20]:
import numpy as np
from numpy.linalg import norm

model_name = "jinaai/jina-embeddings-v2-small-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    return embedding[0].numpy()

# Get embeddings
q_vec = get_embedding("I just discovered the course. Can I join now?")
doc_vec = get_embedding("Can I still join the course after the start date?")

# Cosine similarity = dot product since vectors are normalized
q_vec = q_vec / norm(q_vec)
doc_vec = doc_vec / norm(doc_vec)

cosine_similarity = np.dot(q_vec, doc_vec)
print(cosine_similarity)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

0.9017081


In [21]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [22]:
model_name = "jinaai/jina-embeddings-v2-small-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    return embedding[0].numpy()

documents = [
    {"text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."},
    {"text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project."},
    {"text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."},
    {"text": "You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects."},
    {"text": "Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository."}
]

query = "I just discovered the course. Can I join now?"

doc_embeddings = np.array([get_embedding(doc["text"]) for doc in documents])
query_embedding = get_embedding(query)

doc_embeddings = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)
query_embedding = query_embedding / np.linalg.norm(query_embedding)

similarities = doc_embeddings.dot(query_embedding)

for idx, sim in enumerate(similarities):
    print(f"Document {idx} similarity: {sim:.3f}")

best_index = np.argmax(similarities)
print(f"\n Most similar document index: {best_index}")

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Document 0 similarity: 0.713
Document 1 similarity: 0.732
Document 2 similarity: 0.635
Document 3 similarity: 0.705
Document 4 similarity: 0.792

 Most similar document index: 4


In [23]:
documents = [
    {
        "question": "Course - Can I still join the course after the start date?",
        "text": "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."
    },
    {
        "question": "Course - Can I follow the course after it finishes?",
        "text": "Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project."
    },
    {
        "question": "Course - When will the course start?",
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel."
    },
    {
        "question": "Course - What can I do before the course starts?",
        "text": "You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects."
    },
    {
        "question": "How can we contribute to the course?",
        "text": "Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository."
    }
]

query = "I just discovered the course. Can I join now?"

query_vec = get_embedding(query)
query_vec = query_vec / np.linalg.norm(query_vec)

full_texts = [doc["question"] + " " + doc["text"] for doc in documents]
doc_embeddings = np.array([get_embedding(text) for text in full_texts])
doc_embeddings = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)

similarities = doc_embeddings.dot(query_vec)

for i, sim in enumerate(similarities):
    print(f"Document {i} similarity: {sim:.3f}")

best_index = np.argmax(similarities)
print(f"\n Most similar document index: {best_index}")

Document 0 similarity: 0.673
Document 1 similarity: 0.720
Document 2 similarity: 0.649
Document 3 similarity: 0.741
Document 4 similarity: 0.740

 Most similar document index: 3


In [27]:
# q5
import json
from fastembed import TextEmbedding
TextEmbedding.list_supported_models()

for model in TextEmbedding.list_supported_models():
    if model["dim"] < 500:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
{
  "model": "snowflake/snowflake-arctic-embed-xs",
  "sou

In [28]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
documents_raw = requests.get(docs_url).json()

documents = []
for course in documents_raw:
    if course['course'] != 'machine-learning-zoomcamp':
        continue
    for doc in course['documents']:
        doc['course'] = course['course']
        documents.append(doc)

model_handle = "BAAI/bge-small-en"
EMBEDDING_DIM = 384

collection_name = "ml-zoomcamp-faq"
client = QdrantClient("http://localhost:6333")

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIM,
        distance=models.Distance.COSINE,
    )
)

points = []
for i, doc in enumerate(documents):
    full_text = doc['question'] + " " + doc['text']
    vector = models.Document(text = text, model = model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

client.upsert(
    collection_name=collection_name,
    points=points
)

query = "I just discovered the course. Can I join now?"

results = client.search(
    collection_name=collection_name,
    query_vector=query,
    limit=5,
    with_payload=True
)
print(f"Top similarity score: {results[0].score:.4f}")

/var/folders/95/00r3y4ys0cxgk7xbsyl1smv00000gn/T/ipykernel_69953/1686003113.py:23: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
Fetching 5 files: 100%|██████████| 5/5 [00:02<00:00,  2.44it/s]
/var/folders/95/00r3y4ys0cxgk7xbsyl1smv00000gn/T/ipykernel_69953/1686003113.py:53: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


ValidationError: 3 validation errors for SearchRequest
vector.list[float]
  Input should be a valid list [type=list_type, input_value='I just discovered the course. Can I join now?', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type
vector.NamedVector
  Input should be a valid dictionary or instance of NamedVector [type=model_type, input_value='I just discovered the course. Can I join now?', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type
vector.NamedSparseVector
  Input should be a valid dictionary or instance of NamedSparseVector [type=model_type, input_value='I just discovered the course. Can I join now?', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [30]:
from fastembed.embedding import DefaultEmbedding
embedding_model = DefaultEmbedding(model_name="BAAI/bge-small-en")
EMBEDDING_DIM = 384
collection_name = "ml-zoomcamp-faq"
client = QdrantClient("http://localhost:6333")

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIM,
        distance=models.Distance.COSINE
    )
)

points = []
for i, doc in enumerate(documents):
    full_text = doc['question'] + " " + doc['text']
    # Embed the full_text (returns a generator, get first vector)
    vector = next(embedding_model.embed([full_text]))

    point = models.PointStruct(
        id=i,
        vector=vector.tolist(),  # convert numpy array to list if needed
        payload=doc
    )
    points.append(point)

client.upsert(collection_name=collection_name, points=points)
query = "I just discovered the course. Can I join now?"
query_vector = next(embedding_model.embed([query]))
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector.tolist(),
    limit=5,
    with_payload=True
)
print(f"Top similarity score: {results[0].score:.4f}")

2025-07-01 18:56:30.224 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.
/var/folders/95/00r3y4ys0cxgk7xbsyl1smv00000gn/T/ipykernel_69953/695693142.py:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Top similarity score: 0.8703


/var/folders/95/00r3y4ys0cxgk7xbsyl1smv00000gn/T/ipykernel_69953/695693142.py:42: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
